In [11]:
from secrets import token_bytes
from hashlib import sha1
import base64

# THE FATE OF YOUR GRADE HANGS IN THE BALANCE...
# PROFESSOR'S CHALLENGE ECHOES IN YOUR MIND: "BREAK THIS CIPHER, GET 4.0!"

def pad(data_to_pad, block_size):
    # PADDING: BECAUSE EVEN REBELS NEED TO FOLLOW SOME RULES
    padding_len = block_size - len(data_to_pad) % block_size
    padding = bytes([padding_len] * padding_len)
    return data_to_pad + padding


def crypt(data: bytes, key: bytes, mode: str, rounds: int):
    # THE REBEL'S MASTERPIECE: DES CORE MUTILATED WITH SHA1 HEART TRANSPLANT
    # BLOCK SIZE: 4 BYTES (BECAUSE WHO NEEDS STANDARDS ANYWAY?)
    # KEY SIZE: 6 BYTES, 48 BITS (COMPROMISE IS THE NAME OF THE GAME)

    assert len(key) == 6  # THE CHAINS OF CONVENTION
    assert len(data) % 4 == 0  # CONFORMITY IN REBELLION
    assert mode == "e" or mode == "d"  # ENCRYPT OR DECRYPT? THE ETERNAL QUESTION

    res = bytearray()
    keys = [
        key[0:3],  # HALF A KEY FOR TWICE THE FUN
        key[3:6],  # THE OTHER HALF OF THIS DISASTER
    ]

    for i in range(0, len(data), 4):
        part = data[i : i + 4]
        L = part[0:2]  # LEFT HALF: INNOCENT BYSTANDER
        R = part[2:4]  # RIGHT HALF: ABOUT TO GET SHA1-MASHED

        for r in range(rounds):
            if mode == "e":
                round_key = keys[r % 2]  # KEY SCHEDULE: TOO SIMPLE TO FAIL?
            else:
                round_key = keys[
                    (r + 1) % 2
                ]  # DECRYPTION: WALKING BACKWARDS THROUGH CHAOS

            # THE MOMENT OF TRUTH: SHA1 AS FEISTEL FUNCTION
            # THIS IS WHERE THE REBEL'S DREAM MEETS CRYPTOGRAPHIC REALITY
            temp = sha1(R + round_key).digest()  # HASHING OUR WAY TO GLORY (OR RUIN)

            # THE FEISTEL DANCE: SWAP AND MUTATE
            L, R = R, bytes(
                [a ^ b for a, b, in zip(L, temp)]
            )  # XOR: THE BUTTERFLY EFFECT

        enc = R + L  # FINAL SWAP: THE GRAND ILLUSION
        res += enc  # COLLECTING THE PIECES OF OUR BROKEN DREAMS

    return bytes(res)  # BEHOLD: THE MONSTROSITY IN ALL ITS GLORY


def encrypt(data: bytes, key: bytes):
    # ENTER THE DRAGON: 32 ROUNDS OF PSEUDO-SECURITY
    return crypt(data, key, "e", 32)


def decrypt(data: bytes, key: bytes):
    # REVERSE THE CURSE: CAN WE UNDO THIS MADNESS?
    return crypt(data, key, "d", 32)


In [ ]:
# THE ARENA: WHERE GRADES ARE WON AND DREAMS DIE
mode = input("easy or hard?")  # CHOOSE YOUR FATE, BRAVE SOUL

if mode == "easy":
    # EASY MODE: THE PROFESSOR'S WARM-UP LAUGH
    key = token_bytes(6)  # RANDOM KEY: THE DICE OF DESTINY
    flag_easy = open("/flag_easy", encoding="utf-8").read().strip()
    plain = pad(base64.b16encode(flag_easy.encode()), 4)
    enc_flag = encrypt(plain, key)  # THE FIRST VICTIM OF OUR CRYPTO-EXPERIMENT
    assert (
        decrypt(enc_flag, key) == plain
    )  # SANITY CHECK: EVEN MAD SCIENTISTS HAVE STANDARDS
    print(enc_flag.hex())  # THE CHALLENGE: CRACK THIS AND CLAIM YOUR REWARD

    # THE TRAINING GROUNDS: PRACTICE MAKES PERFECT (OR BREAKS SPIRITS)
    # HINT: YOU CAN SEND DATA IN BATCH TO SPEED UP IO
    while True:
        plain = bytes.fromhex(input())
        print(encrypt(plain, key).hex())  # THE ORACLE SPEAKS: ENCRYPTION ON DEMAND

elif mode == "hard":
    # HARD MODE: THE PROFESSOR'S FINAL LAUGH
    key = token_bytes(6)  # ANOTHER RANDOM KEY: DOUBLE THE TROUBLE
    flag_hard = open("/flag_hard", encoding="utf-8").read().strip()
    flag_hard_padded = pad(flag_hard.encode(), 4)
    xor_key = token_bytes(len(flag_hard_padded))  # EXTRA LAYER OF CONFUSION
    scrambled = bytes(
        [a ^ b for a, b in zip(flag_hard_padded, xor_key)]
    )  # MIXING THE POT

    # DOUBLE ENCRYPTION: BECAUSE ONE LAYER OF INSANITY WASN'T ENOUGH
    enc_scrambled = encrypt(scrambled, key)  # THE MAZE
    enc_xor_key = encrypt(xor_key, key)  # THE KEY TO THE MAZE (ENCRYPTED, OF COURSE)
    print(enc_scrambled.hex())  # THE FIRST CLUE IN THIS TREASURE HUNT
    print(enc_xor_key.hex())  # THE SECOND CLUE: NOW PUT THEM TOGETHER

    # THE GAUNTLET: 100,000 CHANCES TO PROVE YOUR WORTH
    # HINT: YOU CAN SEND MULTIPLE LINES BEFORE WAITING FOR RESPONSE TO SPEED UP IO
    for i in range(100000):
        plain = bytes.fromhex(input())
        print(encrypt(plain[:4], key).hex())  # THE ORACLE WHISPERS: ONE BLOCK AT A TIME

# THE CLOCK IS TICKING... CAN YOU BREAK THE REBEL'S CIPHER BEFORE YOUR GRADE SHATTERS?
# THE PROFESSOR WATCHES, AMUSED. THE BALL IS IN YOUR COURT.


## flag1

In [120]:
from pwn import *

r = remote('prob12.geekgame.pku.edu.cn', 10012)
r.recv()
r.sendline(b'GgT-8j-Edq1WkAHBzfYemt1HGSx2N7KlXfEbiu2I03lne-RS563K0ALLvJL5SAs_92ikd9pc9qmr0Rf6ky71vkf3DwgE')
r.recv()
r.sendline(b'easy')
r.recvline()

[x] Opening connection to prob12.geekgame.pku.edu.cn on port 10012
[x] Opening connection to prob12.geekgame.pku.edu.cn on port 10012: Trying 115.27.246.20
[+] Opening connection to prob12.geekgame.pku.edu.cn on port 10012: Done


b'0771941b15f4f52963159ae683599deec3188131ef7f92eb3f08c9a2e3deca110f1f1d327c97a7988e6b666f92ec56d85e103d74f1c1f7b7dfc7111913d8f7f11209d0d68e6b666f92294a321303a506a9bae27c6d9a0dd5abfea5fb053737f6990aac4e87b9ea77\n'

In [128]:
from tqdm import tqdm

ALPHABET = "0123456789ABCDEF"

# 构建明密文映射字典, bytes -> hex
plain_to_cipher = {}
cipher_to_plain = {}
        
total_possible = 16**4
batch_size = 4096
all_plaintexts = []
for n in range(total_possible):
    candidate = ""
    temp = n
    for j in range(4):
        digit = temp % 16
        candidate += ALPHABET[digit]
        temp //= 16
    all_plaintexts.append(candidate.encode())
    
# 批量查询Oracle
for batch_start in tqdm(range(0, total_possible, batch_size)):
    batch_end = min(batch_start + batch_size, total_possible)
    batch_plaintexts = all_plaintexts[batch_start: batch_end]

    # 构建批量查询的长字符串
    batch_data = b"".join(batch_plaintexts)

    # 批量加密
    try:
        r.sendline(batch_data.hex().encode())
        batch_cipher = r.recvline().decode().strip()
        print(len(batch_cipher))
    except Exception as e:
        print(f"加密失败: {e}")
        continue

    # 分段处理结果，每4字节对应一个明文块
    for i in range(batch_size):
        plain_block = batch_data[i*4: i*4+4]
        cipher_block = batch_cipher[i*8: i*8+8]
        
        # 存储映射关系
        plain_to_cipher[plain_block] = cipher_block
        cipher_to_plain[cipher_block] = plain_block

 12%|█▎        | 2/16 [00:00<00:02,  6.34it/s]

32768
32768


 25%|██▌       | 4/16 [00:00<00:01,  6.39it/s]

32768
32768


 38%|███▊      | 6/16 [00:00<00:01,  6.30it/s]

...

32768
32768


 88%|████████▊ | 14/16 [00:02<00:00,  6.40it/s]

32768
32768


100%|██████████| 16/16 [00:02<00:00,  6.36it/s]

32768
32768


In [130]:
enc_flag = "0771941b15f4f52963159ae683599deec3188131ef7f92eb3f08c9a2e3deca110f1f1d327c97a7988e6b666f92ec56d85e103d74f1c1f7b7dfc7111913d8f7f11209d0d68e6b666f92294a321303a506a9bae27c6d9a0dd5abfea5fb053737f6990aac4e87b9ea77"

In [139]:
plain = ""
for i in range(0, len(enc_flag), 8):
    tmp = enc_flag[i: i+8]
    try:
        plain += cipher_to_plain[tmp].decode()
    except:
        break # 忽略结尾的padding

In [135]:
bytes.fromhex(plain)

b'flag{sHORT_BLOCK_sIzE_IS_VuLNERAblE_TO_brUtEfORCe}'

## flag2

In [301]:
from pwn import *

r = remote('prob12.geekgame.pku.edu.cn', 10012)
r.recv()
r.sendline(b'GgT-8j-Edq1WkAHBzfYemt1HGSx2N7KlXfEbiu2I03lne-RS563K0ALLvJL5SAs_92ikd9pc9qmr0Rf6ky71vkf3DwgE')
r.recv()
r.sendline(b'hard')
print(r.recvline())
print(r.recvline())

[x] Opening connection to prob12.geekgame.pku.edu.cn on port 10012
[x] Opening connection to prob12.geekgame.pku.edu.cn on port 10012: Trying 115.27.246.20
[+] Opening connection to prob12.geekgame.pku.edu.cn on port 10012: Done
b'c277ea842f8f07bb4ab29e90c5edc378acc1209fab9ff05ce0cb9be6ca2878f83de8c10799a29ce0e46f20b80eeb143a\n'
b'14f31dea86473b80023a46d1f329baa7477d1fb12f51f0873bb65d71b80efa6ff7e479f96cd47586aa7e2acf791bff1c\n'


In [302]:
# L0, R0 -> enc -> R32, L32
# L1, R1 -> dec -> R33, L33

key = token_bytes(6)
def oracle(bt, local=False, key=None):
    if local:
        return encrypt(bt, key) 
    else:
        r.sendline(bt.hex().encode())
        v = r.recvline().strip()
        return bytes.fromhex(v.decode())

P0 = b"flag"
L0, R0 = P0[:2], P0[2:]
enc = oracle(P0)
R32, L32 = enc[:2], enc[2:]
L33 = R32
for n in tqdm(range(2**16)):
    R33 = n.to_bytes(2, byteorder='big')
    P1 = oracle(R33 + L33)
    if len(P1) != 4:
        print("Error!", P1)
        break
    if P1[:2] == R0:
        print("Successed!", P1, R33+L33)

 72%|███████▏  | 47246/65536 [00:26<00:10, 1796.09it/s]

Successed! b'ag\x95\xe3' b'\xb7\xc7\xceT'


100%|██████████| 65536/65536 [00:35<00:00, 1820.88it/s]


In [303]:
P0 = L0 + R0
P1 = b'ag\x95\xe3'
P32 = enc
P33 = b'\xb7\xc7\xceT'
possible_k1 = []
possible_k2 = []

In [304]:
P0,P1,P32,P33

(b'flag', b'ag\x95\xe3', b'\xceT\x9b\xd9', b'\xb7\xc7\xceT')

In [305]:
def one_round(L, R, k):
    temp = sha1(R + k).digest() 
    L, R = R, bytes(
        [a ^ b for a, b, in zip(L, temp)]
    )
    return L + R
    
for k in tqdm(range(2**24)):
    k1 = k.to_bytes(3, byteorder='big')
    if one_round(L0, R0, k1) == P1 and one_round(P32[2:], P32[:2], k1) == P33[2:] + P33[:2]:
        print("Correct key 1:", k)

 19%|█▉        | 3176033/16777216 [00:04<00:17, 756692.87it/s]

Correct key 1: 3091475


100%|██████████| 16777216/16777216 [00:22<00:00, 753647.83it/s]


In [306]:
for k in tqdm(range(2**24)):
    k2 = k.to_bytes(3, byteorder='big')
    key = int.to_bytes(3091475, 3, byteorder='big') + k2
    if oracle(P0, local=True, key=key) == P32:
        print("Correct key 2:", k)


 25%|██▍       | 4119354/16777216 [02:09<06:37, 31831.32it/s]

Correct key 2: 4114859


 26%|██▌       | 4283963/16777216 [02:14<06:31, 31900.52it/s]


KeyboardInterrupt: 

In [318]:
key = int.to_bytes(3091475, 3, byteorder='big') + int.to_bytes(4114859, 3, byteorder='big')

In [319]:
oracle(P33, local=True, key=key) == P1

True

In [321]:
enc_scrambled = bytes.fromhex('c277ea842f8f07bb4ab29e90c5edc378acc1209fab9ff05ce0cb9be6ca2878f83de8c10799a29ce0e46f20b80eeb143a')
enc_xor_key = bytes.fromhex('14f31dea86473b80023a46d1f329baa7477d1fb12f51f0873bb65d71b80efa6ff7e479f96cd47586aa7e2acf791bff1c')

scrambled = decrypt(enc_scrambled, key) 
xor_key = decrypt(enc_xor_key, key) 
flag_hard_padded = bytes(
        [a ^ b for a, b in zip(scrambled, xor_key)]
    )
print(flag_hard_padded)

b'flag{sLiDE_ATtaCk_rEaLlY_atTACks_sLIDe_CiPHEr}\x02\x02'
